In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lower_Grade_Glioma"
cohort = "GSE35158"

# Input paths
in_trait_dir = "../../input/GEO/Lower_Grade_Glioma"
in_cohort_dir = "../../input/GEO/Lower_Grade_Glioma/GSE35158"

# Output paths
out_data_file = "../../output/preprocess/Lower_Grade_Glioma/GSE35158.csv"
out_gene_data_file = "../../output/preprocess/Lower_Grade_Glioma/gene_data/GSE35158.csv"
out_clinical_data_file = "../../output/preprocess/Lower_Grade_Glioma/clinical_data/GSE35158.csv"
json_path = "../../output/preprocess/Lower_Grade_Glioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiling of lower-grade diffuse astrocytic glioma"
!Series_summary	"Diffuse gliomas represent the most prevalent class of primary brain tumor.  Despite significant recent advances in the understanding of glioblastoma (WHO IV), its most malignant subtype, lower-grade (WHO II and III) glioma variants remain comparatively understudied, especially in light of their notably variable clinical behavior.  To examine the foundations of this heterogeneity, we performed multidimensional molecular profiling, including global transcriptional analysis, on 101 lower-grade diffuse astrocytic gliomas collected at our own institution, and validated our findings using publically available gene expression and copy number data from large independent patient cohorts.  We found that IDH mutational status delineated molecularly and clinically distinct glioma subsets, with IDH mutant (IDH mt) tumors exhibiting TP53 mutations, PDGFRA overexpression, and prolong

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains expression profiling
# of lower-grade diffuse astrocytic glioma, which indicates gene expression data availability
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Lower_Grade_Glioma), WHO grade could be used as it represents glioma grade
# Index 1 contains WHO grade information
trait_row = 1  

# Age information is not available in the sample characteristics
age_row = None

# Gender information is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert WHO grade information to binary format"""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        grade = value.split(':', 1)[1].strip()
        if grade == 'II':
            return 0  # Lower grade
        elif grade == 'III':
            return 1  # Higher grade
    return None

def convert_age(value):
    """Convert age values to continuous format"""
    # Since age data is not available, define a placeholder function
    return None

def convert_gender(value):
    """Convert gender values to binary format"""
    # Since gender data is not available, define a placeholder function
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Extract clinical features using the library function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # Assumes clinical_data is available from previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save the clinical data as CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM864095': [nan], 'GSM864096': [1.0], 'GSM864097': [1.0], 'GSM864098': [1.0], 'GSM864099': [1.0], 'GSM864100': [1.0], 'GSM864101': [1.0], 'GSM864102': [1.0], 'GSM864103': [1.0], 'GSM864104': [1.0], 'GSM864105': [1.0], 'GSM864106': [1.0], 'GSM864107': [1.0], 'GSM864108': [1.0], 'GSM864109': [1.0], 'GSM864110': [1.0], 'GSM864111': [1.0], 'GSM864112': [1.0], 'GSM864113': [1.0], 'GSM864114': [1.0], 'GSM864115': [1.0], 'GSM864116': [1.0], 'GSM864117': [1.0], 'GSM864118': [1.0], 'GSM864119': [1.0], 'GSM864120': [1.0], 'GSM864121': [1.0], 'GSM864122': [1.0], 'GSM864123': [1.0], 'GSM864124': [1.0], 'GSM864125': [1.0], 'GSM864126': [1.0], 'GSM864127': [0.0], 'GSM864128': [0.0], 'GSM864129': [0.0], 'GSM864130': [0.0], 'GSM864131': [0.0], 'GSM864132': [0.0], 'GSM864133': [0.0], 'GSM864134': [1.0], 'GSM864135': [0.0], 'GSM864136': [0.0], 'GSM864137': [0.0], 'GSM864138': [0.0], 'GSM864139': [1.0], 'GSM864140': [0.0], 'GSM864141': [1.0], 'GSM864142': [1.0], 'GSM864143': 

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Expression profiling of lower-grade diffuse astrocytic glioma"
Line 1: !Series_geo_accession	"GSE35158"
Line 2: !Series_status	"Public on Feb 24 2012"
Line 3: !Series_submission_date	"Jan 17 2012"
Line 4: !Series_last_update_date	"Dec 22 2017"
Line 5: !Series_pubmed_id	"22415316"
Line 6: !Series_summary	"Diffuse gliomas represent the most prevalent class of primary brain tumor.  Despite significant recent advances in the understanding of glioblastoma (WHO IV), its most malignant subtype, lower-grade (WHO II and III) glioma variants remain comparatively understudied, especially in light of their notably variable clinical behavior.  To examine the foundations of this heterogeneity, we performed multidimensional molecular profiling, including global transcriptional analysis, on 101 lower-grade diffuse astrocytic gliomas collected at our own institution, and validated our findings using publically available gene expression and copy 

### Step 4: Gene Identifier Review

In [5]:
# Based on the information provided, I need to determine if gene mapping is needed

# The gene identifiers in the expression data are in the format ILMN_1651209, ILMN_1651228, etc.
# These are Illumina probe IDs, not standard human gene symbols.
# Illumina probe IDs need to be mapped to human gene symbols for more meaningful analysis.

# Illumina IDs typically start with "ILMN_" followed by numbers, which is what we see here.
# These are not directly interpretable as gene names and require mapping to standard gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1713610 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': ['DQ516750', 'DQ883654', 'DQ668364', 'DQ516785', 'DQ854995'], 'Symbol': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which columns to use for mapping
# From the gene annotation data, we need ID and Symbol columns
# ID column in gene_annotation corresponds to ID index in gene_data
# Symbol column contains gene symbols we want to map to

# 2. Get gene mapping dataframe by extracting relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Created gene mapping with {len(gene_mapping)} rows")
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)
print(f"Converted gene expression data from {len(gene_data.index)} genes")
print("First few gene symbols after mapping:")
print(gene_data.index[:20])

# Normalize gene symbols to standard format
print("Normalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene data contains {len(gene_data.index)} unique genes")
print("First few normalized gene symbols:")
print(gene_data.index[:20])

# Save gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Created gene mapping with 29377 rows
Gene mapping preview:
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144']}
Converted gene expression data from 20210 genes
First few gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AADAC', 'AADACL1',
       'AADACL2', 'AADACL4', 'AADAT', 'AAGAB', 'AAK1'],
      dtype='object', name='Gene')
Normalizing gene symbols...


After normalization, gene data contains 19449 unique genes
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AADAC', 'AADACL2', 'AADACL4', 'AADAT',
       'AAGAB', 'AAK1', 'AAMDC', 'AAMP', 'AANAT'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Lower_Grade_Glioma/gene_data/GSE35158.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI Gene database
print("\nNormalizing gene symbols using NCBI Gene database...")
try:
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(gene_data_normalized)} unique genes remain")
    gene_data_cleaned = gene_data_normalized
    
    # Save the normalized gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_cleaned.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene symbol normalization: {str(e)}")
    print("Falling back to original gene data")
    gene_data_cleaned = gene_data.copy()

# 2. We need to recreate the clinical data from the original matrix file
print("\nExtracting clinical data directly from the matrix file...")
# Extract clinical data from the matrix file again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Process clinical data
print("\nProcessing clinical data...")
# Create clinical features dataframe 
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Extracted clinical data with shape: {selected_clinical_df.shape}")
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")
    
    is_trait_available = True
else:
    selected_clinical_df = pd.DataFrame()
    is_trait_available = False
    print("No trait data available in clinical information.")

# 3. Link clinical and genetic data
if is_trait_available and is_gene_available:
    print("\nLinking clinical and genetic data...")
    try:
        # Ensure the sample IDs match between clinical and genetic data
        common_samples = list(set(selected_clinical_df.columns).intersection(set(gene_data_cleaned.columns)))
        
        if len(common_samples) == 0:
            print("Warning: No common samples between clinical and genetic data")
            linked_data = pd.DataFrame()
            is_biased = True
        else:
            print(f"Found {len(common_samples)} common samples between clinical and genetic data")
            
            # Filter data to include only common samples
            clinical_subset = selected_clinical_df[common_samples]
            genetic_subset = gene_data_cleaned[common_samples]
            
            # Link the data
            linked_data = pd.concat([clinical_subset, genetic_subset], axis=0).T
            print(f"Linked data shape: {linked_data.shape}")
            
            # 4. Handle missing values
            print("\nHandling missing values in linked data...")
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Data shape after handling missing values: {linked_data.shape}")
            
            # 5. Determine if trait and demographic features are severely biased
            print("\nEvaluating trait and demographic feature bias...")
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    except Exception as e:
        print(f"Error during data linking: {str(e)}")
        linked_data = pd.DataFrame()
        is_biased = True
else:
    print("\nCannot create linked data: missing clinical or gene data")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final validation and saving
note = "This dataset contains gene expression data from astrocytoma cell lines with modified GFAP isoform expression. The trait represents different experimental conditions related to the GFAPδ/GFAPα ratio."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if len(linked_data) > 0 else True,
    df=linked_data,
    note=note
)

# Save the linked data if it's usable
if is_usable and len(linked_data) > 0:
    print(f"\nSaving linked data to {out_data_file}")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved successfully!")
else:
    print(f"\nDataset not usable for {trait} association studies due to bias or quality issues.")


Normalizing gene symbols using NCBI Gene database...
After normalization: 19449 unique genes remain


Normalized gene expression data saved to ../../output/preprocess/Lower_Grade_Glioma/gene_data/GSE35158.csv

Extracting clinical data directly from the matrix file...

Processing clinical data...
Extracted clinical data with shape: (1, 81)
Clinical data saved to: ../../output/preprocess/Lower_Grade_Glioma/clinical_data/GSE35158.csv

Linking clinical and genetic data...
Found 81 common samples between clinical and genetic data
Linked data shape: (81, 19450)

Handling missing values in linked data...


Data shape after handling missing values: (80, 19450)

Evaluating trait and demographic feature bias...
For the feature 'Lower_Grade_Glioma', the least common label is '0.0' with 29 occurrences. This represents 36.25% of the dataset.
The distribution of the feature 'Lower_Grade_Glioma' in this dataset is fine.


Saving linked data to ../../output/preprocess/Lower_Grade_Glioma/GSE35158.csv


Linked data saved successfully!
